In [1]:
from lxml import etree
import cv2
import numpy as np
import os

In [2]:
def xml_to_coordinates(t):
    result = []
    data=t.split(' ')
    for i,k in zip(data[0::2], data[1::2]):
        result.append((int(i),int(k)))
    result=np.array(result)
    return result

In [3]:
def centroid_from_coordinates(coords):
    #the coords are ordered as counter clock wise
    cx=int(coords[0][0]+(coords[2][0]-coords[0][0])/2)
    cy=int(coords[1][1]+(coords[3][1]-coords[1][1])/2)
    return (cx,cy)

In [4]:
def inner_polygon_by_threshold(coords,thresh):
    
    x0=coords[0][0]+thresh
    y0=coords[0][1]+thresh
    
    x1=coords[1][0]+thresh
    y1=coords[1][1]-thresh
    
    x2=coords[2][0]-thresh
    y2=coords[2][1]-thresh
    
    x3=coords[3][0]-thresh
    y3=coords[3][1]+thresh
    
    inner_polygon=[[x0,y0],[x1,y1],[x2,y2],[x3,y3]]
    
    inner_polygon=np.array(inner_polygon)
    
    return inner_polygon

In [5]:
def inner_polygon_by_percentage(coords,percent):
    
    x0=coords[0][0]
    y0=coords[0][1]
    
    x1=coords[1][0]
    y1=coords[1][1]
    
    x2=coords[2][0]
    y2=coords[2][1]
    
    x3=coords[3][0]
    y3=coords[3][1]
    
    ly=abs(y1-y0)
    my=int(ly*percent/200)
    
    lx=abs(x2-x1)
    mx=int(lx*percent/200)
        
    x0=coords[0][0]+mx
    y0=coords[0][1]+my
    
    x1=coords[1][0]+mx
    y1=coords[1][1]-my
    
    x2=coords[2][0]-mx
    y2=coords[2][1]-my
    
    x3=coords[3][0]-mx
    y3=coords[3][1]+my
    
    inner_polygon=[[x0,y0],[x1,y1],[x2,y2],[x3,y3]]
    
    inner_polygon=np.array(inner_polygon)
    
    return inner_polygon

In [6]:
def create_circular_mask(h, w, center, radius):
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
    mask = dist_from_center <= radius
    return mask

In [ ]:
xml_dir='alto_xmls/'
char_blobs_dir='char_blobs/'
line_blobs_dir='line_blobs/'

for xml_name in os.listdir(xml_dir):
    print(xml_name)
    
    xml_tree = etree.parse(xml_dir+xml_name)
    save_name=xml_name[:-13]+'.png'

    root = xml_tree.getroot()
    for elem in root.getiterator():
        elem.tag = etree.QName(elem).localname
        
    for page in root.findall("Layout/Page"):
        h=int(page.get("HEIGHT"))
        w=int(page.get("WIDTH"))
    
    char_mask = np.zeros([h,w])
    line_mask = np.zeros([h,w])
    
    for line in root.findall("Layout/Page/PrintSpace/TextBlock/TextLine"):  
        for string in line.findall("String"):
            for glyph in string.findall("Glyph"):
                letter = glyph.get("CONTENT")
                if letter!=' ':
                    for shape in glyph.findall("Shape"):
                        for polygon in shape.findall("Polygon"):
                            polygon=polygon.get("POINTS")
                            if len(polygon.split(' '))==8:
                                coords=xml_to_coordinates(polygon)
                                #cv2.drawContours(fragment_image, [coords], -1, (0, 0, 255), 3)
                                cv2.fillConvexPoly(line_mask, np.array(coords, 'int32'), 255)
                                cv2.imwrite(line_blobs_dir+save_name,line_mask)

                                #inner_polygon_coords = inner_polygon(coords,40)
                                #cv2.drawContours(fragment_image, [inner_polygon_coords], -1, (0, 255, 0), 1)
                                #cv2.fillConvexPoly(char_mask, np.array(inner_polygon_coords, 'int32'), 255)

                                centroid=centroid_from_coordinates(coords)
                                #cv2.circle(fragment_image, centroid, 3, (0, 0, 255), -1)
                                cv2.circle(char_mask, centroid, 7, (255, 255, 255), -1)
                                cv2.imwrite(char_blobs_dir+save_name,char_mask)

105-Fg003-R-C01-R01-D03012012-T135030-LR924_012_F.xml_algn.xml
111-Fg005-R-C01-R01-D08112011-T153037-LR924_012_F.xml_algn.xml
1148-Fg008-R-C01-R01-D07022012-T114227-LR924_012_F.xml_algn.xml
115-Fg002-R-C02-R01-D06122011-T084537-LR924_012_F.xml_algn.xml
115-Fg005-R-C01-R01-D06122011-T100445-LR924_012_F.xml_algn.xml
115-Fg006-R-C01-R01-D06122011-T101711-LR924_012_F.xml_algn.xml
124-Fg004-R-C01-R01-D10112011-T105711-LR924_012_F.xml_algn.xml
126-Fg001-R-C01-R01-D13112011-T124909-LR924_012_F.xml_algn.xml
126-Fg002-R-C01-R01-D13112011-T130145-LR924_012_F.xml_algn.xml
128-Fg002-R-C01-R01-D21112011-T144832-LR924_012_F.xml_algn.xml
129-Fg004-R-C01-R01-D22112011-T142914-LR924_012_F.xml_algn.xml
132-Fg001-R-C01-R01-D24112011-T104107-LR924_012_F.xml_algn.xml
137-Fg001-R-C01-R01-D24112011-T123520-LR924_012_F.xml_algn.xml
137-Fg006-R-C01-R01-D24112011-T134152-LR924_012_F.xml_algn.xml
137-Fg007-R-C01-R01-D24112011-T135006-LR924_012_F.xml_algn.xml
144-Fg001-R-C01-R01-D07032012-T085731-LR924_012_F.xml_